In [41]:
import pandas as pd

def func_pars(filepath, sht):  
    df = pd.read_excel(filepath, sheet_name=sht)

    # Evaluar si la hoja es la mensual o es de especialidad, 
    #** Agregar validaciones adicionales para asegurar que solo se procese el file correcto
    if sht == "Mensual":   
        tipo_hoja = "Mensual"
    else:
        tipo_hoja = "Especialidad"

    # Eliminar filas y columnas vacías

    df.dropna(axis = 0,how = "all",inplace = True)  
    df.dropna(axis = 1,how = "all",inplace = True)  
    df = df.reset_index(drop=True)

    # Eliminar fila extra de la hoja "Mensual"

    if tipo_hoja == "Mensual":
        df.drop(2, inplace = True)
        df = df.reset_index(drop=True)

    # La hoja "Especialidad" tiene una columna adicional que permite identificar la tabla donde se encuentra el agregado de datos 
    # Se eliminan las otras filas y se actualiza el df al tamaño correcto para procesarlo

    elif tipo_hoja == "Especialidad":
        indexnum = int(df.loc[df.iloc[:,0] == "TOTAL"].index[0])

        df = df.iloc[indexnum:]

        nombre_esp = df.iloc[0,1]
        codigo_esp = sht  #depende de cada hoja
        df = df.iloc[:,1:] 

    # Generar una tabla(Posible serie), de las horas contratadas, programadas, etc.
    # Se hace un slice de las primeras dos filas, y se eliminan vacios 

    df_jornada = df.iloc[1:3,1:8].copy() 
    df_jornada.dropna(axis = 1,how = "all",inplace = True)
    df_jornada.columns = df_jornada.iloc[0]
    df_jornada = df_jornada[1:]
    df_jornada = df_jornada.reset_index(drop=True)
    df_jornada.columns.rename("",inplace=True)
    df_jornada = df_jornada.transpose()
    df_jornada.rename(columns={0: 'Horas'},inplace= True)

    # Replicar la tabla de excel con las horas utilizadas por actividad
    # Se hace un slice de las primeras filas 3 a 17  , se eliminan vacios y se nombran columnas de acuerdo al excel

    df_tiempo = df.iloc[3:18].copy()
    df_tiempo.dropna(axis = 1,how = "all",inplace = True)
    df_tiempo.dropna(axis = 0,how = "all",inplace = True)
    df_tiempo.columns = ['Actividad', 'Programado', 'Utilizado', 'N° Actividades']
    df_tiempo = df_tiempo.reset_index(drop=True)

    # Replicar la tabla de excel del resumen del mes con los totales de consultas por cada categoria

    # Se hace un slice de las primeras filas 18 en adelante, se eliminan vacios full y se auto completan algunas categorias faltantes
    df_total = df.iloc[18:].copy()
    df_total.dropna(axis = 1,how = "all",inplace = True)
    df_total[:3].fillna(method="ffill", axis=1,inplace = True)
    df_total[:3].fillna(method="ffill", axis=0,inplace = True)

    # Se transpone la tabla para concatenar las categorias y unificarlas en una sola columna(celdas combinadas de excel) 
    df_total = df_total.transpose()
    df_total.iloc[1, 1] = df_total.iloc[1, 1] +" - "+ df_total.iloc[1, 2]
    df_total.iloc[2:6, 1] = df_total.iloc[2:6, 1].astype(str) +" - "+ df_total.iloc[2:6, 2].astype(str) +" - "+ df_total.iloc[2:6, 3].astype(str)
    df_total.iloc[1:-3, 0] = df_total.iloc[1:-3, 0].astype(str) +" - "+ df_total.iloc[1:-3, 1].astype(str)

    #Se eliminan las columnas que tenian las subcategorias, y se reindexa
    df_total.drop(df_total.columns[[1, 2, 3]],axis = 1, inplace = True)
    df_total.columns = df_total.iloc[0]
    df_total = df_total[1:]
    df_total = df_total.reset_index(drop=True)
    df_total.columns.rename("",inplace=True)
    df_total.iloc[:, 0] = df_total.iloc[:, 0].str.replace('Frecuencia (marcar solo una)', 'Frecuencia', regex=False)

    # Se transpone la tabla para volver al formato original (validar si vale la pena)
    df_total = df_total.transpose()
    df_total.columns = df_total.iloc[0]
    df_total = df_total[1:]

    return {
        'df_jornada': df_jornada,
        'df_tiempo': df_tiempo,
        'df_total': df_total
        }

In [43]:
import pandas as pd
pd.set_option('mode.chained_assignment', None)
filepath = "Prueba.xlsx"

xls = pd.ExcelFile(filepath)

sheet_names = xls.sheet_names

result_dataframes = {}

for sht in sheet_names:    
    result_dataframes[sht] = func_pars(filepath, sht)




In [48]:
result_dataframes["ESP. ABC"]["df_tiempo"]

,Actividad,Programado,Utilizado,N° Actividades
0,Actividad administrativa,0,0,0
1,Alimentación,0,0,NaN
2,Capacitación,0,0,0
3,Comisión,0,0,0
4,Consulta Externa,0,0,NaN
5,Docencia,0,0,0
6,Emergencias,0,0,0
7,Esterilización,0,0,0
8,Hospitalización,0,0,0
9,Prevención colectiva,0,0,0
